# BatchStudy

In this notebook, we explain how different models, experiments, chemistries, etc. can be compared with each other using the `BatchStudy` class.

## Comparing models
We start by creating a simple script to compare `SPM`, `SPMe` and `DFN` model with the default parameters.

In [1]:
# %pip install pybamm -q    # install PyBaMM if it is not installed
import pybamm

# load up 3 models to compare
dfn = pybamm.lithium_ion.DFN()
spm = pybamm.lithium_ion.SPM()
spme = pybamm.lithium_ion.SPMe()

The `BatchStudy` class requires a dictionary of models, and all the default values for a given model are used if no additional parameter is passed in.

In [2]:
models = {
    "dfn": dfn,
    "spm": spm,
    "spme": spme,
}

# creating a BatchStudy object
batch_study = pybamm.BatchStudy(models=models)

# solving and plotting the comparison
batch_study.solve(t_eval=[0, 3600])
batch_study.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

`BatchStudy` by default requires equal number of items in all the dictionaries passed, which can be changed by setting the value of `permutations` to `True`. When set `True`, a cartesian product of all the available items is taken.

For example, here we pass 3 models but only 1 parameter value, hence it is necessary to set `permutations` to `True`. Here, the given parameter value is used for all the provided models.

In [3]:
# passing parameter_values as a dictionary
chen2020 = pybamm.parameter_sets.Chen2020
parameter_values = {"Chen2020": pybamm.ParameterValues(chemistry=chen2020)}

# creating a BatchStudy object and solving the simulation
batch_study = pybamm.BatchStudy(models=models, parameter_values=parameter_values, permutations=True)
batch_study.solve(t_eval=[0, 3600])
batch_study.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=3566.800904290021, step=35.66800904290021), …

## Comparing parameters

`BatchStudy` can also be used to compare different things (like effect of changing a parameter's value) on a single model.

In the following cell, we compare different values of `"Curent function [A]"` using the `Single Paritcle Model with electrolyte`.


In [4]:
model = {"spme": spme}

# populating a dictionary with 3 same parameter values
parameter_values = {
    "Chen2020_1": pybamm.ParameterValues(chemistry=chen2020),
    "Chen2020_2": pybamm.ParameterValues(chemistry=chen2020),
    "Chen2020_3": pybamm.ParameterValues(chemistry=chen2020),
}

# different values for "Current function [A]"
current_values = [4.5, 4.75, 5]

# changing the value of "Current function [A]" in all the parameter values present in the 
# parameter_values dictionary
for k, v, current_value in zip(parameter_values.keys(), parameter_values.values(), current_values):
    v["Current function [A]"] = current_value 

# creating a BatchStudy object with permutations set to True to create a cartesian product
batch_study = pybamm.BatchStudy(models=model, parameter_values=parameter_values, permutations=True)
batch_study.solve(t_eval=[0, 3600])

# generating the required labels and plotting
labels = [f"Current function [A]: {current}" for current in current_values]
batch_study.plot(labels=labels)

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

## Using experiments



In [ ]:
pybamm.set_logging_level("NOTICE")

cccv = pybamm.Experiment(
    [
        ("Discharge at C/10 for 10 hours or until 3.3 V",
        "Rest for 1 hour",
        "Charge at 1 A until 4.1 V",
        "Hold at 4.1 V until 50 mA",
        "Rest for 1 hour")
    ]
    * 10,
)

mohtat2020 = pybamm.parameter_sets.Mohtat2020
parameter_values = {
    "Mohtat2020_1": pybamm.ParameterValues(chemistry=mohtat2020),
    "Mohtat2020_2": pybamm.ParameterValues(chemistry=mohtat2020),
    "Mohtat2020_3": pybamm.ParameterValues(chemistry=mohtat2020),
}

inner_sei_oc_v = [2.0e-4, 2.7e-4, 3.4e-4]

for k, v, sei_kin_rate_const in zip(parameter_values.keys(), parameter_values.values(), current_values):
    v.update(
        {
            "Inner SEI open-circuit potential [V]": inner_sei_oc_v
        },
    )

model = {"spm": pybamm.lithium_ion.SPM({"SEI": "electron-migration limited"})}

experiment = {
    "cccv": cccv
}

batch_study = pybamm.BatchStudy(models=model, experiments=experiment, parameter_values=parameter_values, permutations=True)
batch_study.solve(initial_soc=1)
batch_study.plot()


In [ ]:
pybamm.plot_summary_variables([sim.solution for sim in batch_study.sims])